<a id="primero"></a>
## 1. Calidad de un vino
---

Existen muchas variedades de vino existentes debido a los distintos gustos que tienen las personas. Del gusto se desprende la calidad que una persona le podría asignar a un vino, el cual proviene del gusto de la persona en particular, o bien, a la gran cantidad de quı́micos y procesos que se aplican a la producción de vino. Para el área de negocios, el estimar cuál es la calidad de un vino en base a la apreciación del público es una tarea bastante difı́cil.  
Para esta actividad se trabajará con dos *datasets* asociados a las variantes tinto y blanco del vino portugués
”Vinho Verde”[[1]](#refs). Debido a temas privados solo se cuenta con las caracterı́stcas fisioquı́micas asociadas a un
vino en particular, los cuales corresponden a 11 atributos numéricos descritos en el siguiente __[link](http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality.names)__.

Este problema puede ser abordado como clasificación de 11 clases o de regresión, ya que el atributo a estimar,
*quality*, tiene un dominio como valor entero 0 y 10. La forma de resolverlo será a través de **ensamblados**.

<img src="https://uploads.toptal.io/blog/image/92064/toptal-blog-image-1454584112948-fc1d35939aa1886bf30c816b3ac20e21.jpg" title="Title text" width="20%"  />


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier

> a) Carge los dos dataset en un único dataframe de pandas, además de agregar una columna indicando si es vino tinto o blanco. Describa el dataset a trabajar.

In [3]:
import pandas as pd
df_red = pd.read_csv("winequality-red.csv",sep=";").assign(white= lambda x: 0, red = lambda x: 1)
df_white = pd.read_csv("winequality-white.csv",sep=";").assign(white= lambda x: 1, red = lambda x: 0)
df = pd.concat([df_red,df_white], axis=0)

In [4]:
df.head(3)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,red,white
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,1,0


In [5]:
from sklearn.utils import shuffle
df = shuffle(df)

In [6]:
df.head(3)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,red,white
1319,6.4,0.25,0.53,6.6,0.038,59.0,234.0,0.9955,3.03,0.42,8.8,5,0,1
2023,6.3,0.21,0.28,1.5,0.051,46.0,142.0,0.9928,3.23,0.42,10.1,6,0,1
1577,7.1,0.64,0.49,1.8,0.050,17.0,128.0,0.9946,3.31,0.58,10.6,4,0,1


In [7]:
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,red,white
count,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000
mean,7.215307,0.339666,0.318633,5.443235,0.056034,30.525319,115.744574,0.994697,3.218501,0.531268,10.491801,5.818378,0.246114,0.753886
std,1.296434,0.164636,0.145318,4.757804,0.035034,17.749400,56.521855,0.002999,0.160787,0.148806,1.192712,0.873255,0.430779,0.430779
min,3.800000,0.080000,0.000000,0.600000,0.009000,1.000000,6.000000,0.987110,2.720000,0.220000,8.000000,3.000000,0.000000,0.000000
25%,6.400000,0.230000,0.250000,1.800000,0.038000,17.000000,77.000000,0.992340,3.110000,0.430000,9.500000,5.000000,0.000000,1.000000
50%,7.000000,0.290000,0.310000,3.000000,0.047000,29.000000,118.000000,0.994890,3.210000,0.510000,10.300000,6.000000,0.000000,1.000000
75%,7.700000,0.400000,0.390000,8.100000,0.065000,41.000000,156.000000,0.996990,3.320000,0.600000,11.300000,6.000000,0.000000,1.000000
max,15.900000,1.580000,1.660000,65.800000,0.611000,289.000000,440.000000,1.038980,4.010000,2.000000,14.900000,9.000000,1.000000,1.000000


I) Se pueden observar las siguientes carácterísticas químicas de los vinos (inputs):
    1) fixed acidity
    2) volatile acidity
    3) citric acid
    4) residual sugar
    5) chlorides
    6) free sulfur dioxide
    7) total sulfur dioxide
    8) density
    9) pH
    10) sulphates
    11) alcohol
    12) red
    13) white
II) Se puede observar la característica que se quiere predecir (output):
    1) quality: el mínimo es 3 y el máximo es 9

El dataframe está muy ordenado. Arriba quedaron todos los vinos rojos y abajo todos los blancos. La razón de lo anterior es el mecanísmo de concatenado

>b) Aborde este problema como si fuera de clasificación con multiples clases para predecir el valor de calidad de un vino, es decir, utilice las distintas caracterı́sticas fisioquı́micas presentes en los datos para estimar la etiqueta ¿Cuántas clases son y cuántos ejemplos hay por clase? ¿Qué sucede con predecir si un vino tiene calidad mínima (0) o máxima(10)? Además para el propósito académico de esta actividad cree un conjunto de pruebas (20%) para evaluar la generalización final del modelo y otro de validación (20%) si estima conveniente.

In [8]:
print df.quality.nunique()
print df.quality.count()

7
6497


In [9]:
df.quality.value_counts()

6    2836
5    2138
7    1079
4     216
8     193
3      30
9       5
Name: quality, dtype: int64

Aquí nos damos cuenta que la representación de calidades es precisa. Hay 7 categorías utilizadas.
3, 4, 5, 6, 7, 8, 9

In [10]:
#hay 6497 datos.
import numpy as np
msk = np.random.rand(len(df)) < 0.8
df_train = df[msk]
df_val = df[~msk]

    
x_train = df_train.drop(columns=['quality',])
y_train = df_train["quality"]

x_test = df_val.drop(columns=['quality',])
y_test = df_val["quality"]
#y = df["quality"].values

Importante notar que se separaron random los conjuntos de entrenamiento y de prueba, logrando evitar que todos los blancos y tintos queden juntos entre ellos.

In [11]:
print x_train.count()
print y_train.count()
print "**********"
print x_test.count()
print y_test.count()

fixed acidity           5193
volatile acidity        5193
citric acid             5193
residual sugar          5193
chlorides               5193
free sulfur dioxide     5193
total sulfur dioxide    5193
density                 5193
pH                      5193
sulphates               5193
alcohol                 5193
red                     5193
white                   5193
dtype: int64
5193
**********
fixed acidity           1304
volatile acidity        1304
citric acid             1304
residual sugar          1304
chlorides               1304
free sulfur dioxide     1304
total sulfur dioxide    1304
density                 1304
pH                      1304
sulphates               1304
alcohol                 1304
red                     1304
white                   1304
dtype: int64
1304


No se podrán predecir vinos de calidad mínima (0) ni máxima (10) ya que la máquina no tiene manera de aprender que existen. 
El modelo se puede reparar fácilmente usando algún dataset extra que permita incluir en el modelo de aprendizaje los parámetros que definen un vino como 0, 1, 2 o 10 como categoría.
Por otro lado si obtuvieramos de alguna manera la combinación de parámetros que definen un vino perfecto (10) y uno malo (0) podríamos generar en base a calculos estadísticos una aproximación de modelo.

>c) Entrene un solo Árbol de Clasificación de múltiples niveles para resolver el problema. Defina un Árbol no regularizado (como el que no tiene límites en su profundidad) y otro Árbol regularizado (variando los hiper-parámetros que prefiera, por ejemplo, los más comunes como la profundidad, el número mínimo de datos para realizar split o el número mínimo de datos en cada hoja), recuerde que las decisiones no pueden ser basadas mirando el conjunto de pruebas. Debido al desbalanceo que se produce en las clases mida la métrica F1-score [2] sobre el conjunto de entrenamiento y de pruebas.

In [12]:
from sklearn.tree import DecisionTreeClassifier as Tree
modelT = Tree(max_depth=10000) 
modelT.fit(x_train,y_train.values)
PrediccionT=modelT.predict(x_test)
print("f1.score: ",f1_score(y_true=y_test,y_pred=PrediccionT,average="micro"))

('f1.score: ', 0.6035276073619632)


>d) Entrene un ensamblado de árboles de múltiples niveles, mediante la técnica de Bagging, compare el Árbol no regularizado con el regularizado (seteando los hiper-parámetros en base a lo experimentado anteriormente en c)) ¿Qué debería suceder? ¿Se visualiza overfitting? Varíe la cantidad de árboles de decisión utilizados en el ensamblado (n estimators), realice un gráfico resumen del F1-score de entrenamiento y de pruebas en función de este hiper-parámetro.

In [13]:
y_train.nunique()

7

In [14]:
modelB_1000 = BaggingClassifier(base_estimator=Tree(max_depth=10000), n_estimators=1000, n_jobs=-1)
modelB_1000.fit(x_train,y_train.values)
y_predB_1000 = modelB_1000.predict(x_train)
PrediccionB_1000=modelB_1000.predict(x_test)
print("f1.score train: ",f1_score(y_true=y_train,y_pred=y_predB_1000,average="micro"))
print("f1.score test: ",f1_score(y_true=y_test,y_pred=PrediccionB_1000,average="micro"))

('f1.score train: ', 1.0)
('f1.score test: ', 0.68174846625766872)


In [15]:
#con n=100
modelB_100 = BaggingClassifier(base_estimator=Tree(max_depth=10000), n_estimators=100, n_jobs=-1)
modelB_100.fit(x_train,y_train.values)
y_predB_100 = modelB_100.predict(x_train)
PrediccionB_100=modelB_100.predict(x_test)
print("f1.score train: ",f1_score(y_true=y_train,y_pred=y_predB_100,average="micro"))
print("f1.score test: ",f1_score(y_true=y_test,y_pred=PrediccionB_100,average="micro"))

('f1.score train: ', 1.0)
('f1.score test: ', 0.68328220858895705)


In [16]:
#con n=10
modelB_10 = BaggingClassifier(base_estimator=Tree(max_depth=10000), n_estimators=10, n_jobs=-1)
modelB_10.fit(x_train,y_train.values)
y_predB_10 = modelB_10.predict(x_train)
PrediccionB_10=modelB_10.predict(x_test)
print("f1.score train: ",f1_score(y_true=y_train,y_pred=y_predB_10,average="micro"))
print("f1.score test: ",f1_score(y_true=y_test,y_pred=PrediccionB_10,average="micro"))

('f1.score train: ', 0.98420951280570002)
('f1.score test: ', 0.65414110429447858)


In [17]:
#con n=1
modelB_1 = BaggingClassifier(base_estimator=Tree(max_depth=10000), n_estimators=1, n_jobs=-1)
modelB_1.fit(x_train,y_train.values)
y_predB_1 = modelB_1.predict(x_train)
PrediccionB_1=modelB_1.predict(x_test)
print("f1.score train: ",f1_score(y_true=y_train,y_pred=y_predB_1,average="micro"))
print("f1.score test: ",f1_score(y_true=y_test,y_pred=PrediccionB_1,average="micro"))

('f1.score train: ', 0.84228769497400335)
('f1.score test: ', 0.57055214723926384)


El n estimator más efectivo es n=1000. Se aprecia overfitting para casi todos los n estimators probados.

>e) Entrene un ensamblado de árboles de múltiples niveles, mediante la técnica de AdaBoost, compare el Árbol no regularizado con el regularizado (seteando los hiper-parámetros en base a lo experimentado anteriormente en c) ¿Se visualiza overfitting? ¿Qué técnica se utiliza, re-muestrear o pesar ejemplos? ¿Qué le parece más sensato?. Varíe la cantidad de árboles de decisión utilizados en el ensamblado (n estimators), realice un gráfico resumen del F1-score de entrenamiento y de pruebas en función de este hiper-parámetro. Compare y analice con la técnica utilizada en d).

In [18]:
from sklearn.ensemble import AdaBoostClassifier
modelA_1000 = AdaBoostClassifier(base_estimator=Tree(max_depth=10000), n_estimators=1000)
modelA_1000.fit(x_train,y_train.values)
y_predA_1000= modelA_1000.predict(x_train)
PrediccionA_1000=modelA_1000.predict(x_test)
print("f1.score train: ",f1_score(y_true=y_train,y_pred=y_predA_1000,average="micro"))
print("f1.score test: ",f1_score(y_true=y_test,y_pred=PrediccionA_1000,average="micro"))

('f1.score train: ', 1.0)
('f1.score test: ', 0.60582822085889576)


In [19]:
from sklearn.ensemble import AdaBoostClassifier
modelA_100 = AdaBoostClassifier(base_estimator=Tree(max_depth=10000), n_estimators=100)
modelA_100.fit(x_train,y_train.values)
y_predA_100= modelA_100.predict(x_train)
PrediccionA_100=modelA_100.predict(x_test)
print("f1.score train: ",f1_score(y_true=y_train,y_pred=y_predA_100,average="micro"))
print("f1.score test: ",f1_score(y_true=y_test,y_pred=PrediccionA_100,average="micro"))

('f1.score train: ', 1.0)
('f1.score test: ', 0.60582822085889576)


In [20]:
from sklearn.ensemble import AdaBoostClassifier
modelA_10 = AdaBoostClassifier(base_estimator=Tree(max_depth=10000), n_estimators=10)
modelA_10.fit(x_train,y_train.values)
y_predA_10= modelA_10.predict(x_train)
PrediccionA_10=modelA_10.predict(x_test)
print("f1.score train: ",f1_score(y_true=y_train,y_pred=y_predA_10,average="micro"))
print("f1.score test: ",f1_score(y_true=y_test,y_pred=PrediccionA_10,average="micro"))

('f1.score train: ', 1.0)
('f1.score test: ', 0.60812883435582821)


In [21]:
from sklearn.ensemble import AdaBoostClassifier
modelA_1 = AdaBoostClassifier(base_estimator=Tree(max_depth=10000), n_estimators=1)
modelA_1.fit(x_train,y_train.values)
y_predA_1= modelA_1.predict(x_train)
PrediccionA_1=modelA_1.predict(x_test)
print("f1.score train: ",f1_score(y_true=y_train,y_pred=y_predA_1,average="micro"))
print("f1.score test: ",f1_score(y_true=y_test,y_pred=PrediccionA_1,average="micro"))

('f1.score train: ', 1.0)
('f1.score test: ', 0.6073619631901841)


El n estimator más efectivo es n=1. Se aprecia overfitting para todos los n estimators probados.

>f) Pruebe otra técnica de ensamblado dedicada a árboles de decisión, que combina el muestreo boostrap de Bagging con muestreo sobre las features: Random Forest, compare el Árbol no regularizado con el regularizado ¿Se visualiza overfitting?. Varíe la cantidad de árboles de decisión utilizados en el ensamblado (n estimators), realice un gráfico resumen el F1-score de entrenamiento y de pruebas en función de este hiper-parámetro.

In [22]:
modelF_1000 = RandomForestClassifier(n_estimators=1000, max_depth=10000,n_jobs=-1)
modelF_1000.set_params(warm_start=True,oob_score=True)
modelF_1000.fit(x_train,y_train.values)
y_predF_1000= modelF_1000.predict(x_train)
PrediccionF_1000=modelF_1000.predict(x_test)
print("f1.score train: ",f1_score(y_true=y_train,y_pred=y_predF_1000,average="micro"))
print("f1.score test: ",f1_score(y_true=y_test,y_pred=PrediccionF_1000,average="micro"))

('f1.score train: ', 1.0)
('f1.score test: ', 0.68634969325153372)


In [23]:
from sklearn.ensemble import RandomForestClassifier
modelF_100 = RandomForestClassifier(n_estimators=100, max_depth=10000,n_jobs=-1)
modelF_100.set_params(warm_start=True,oob_score=True)
modelF_100.fit(x_train,y_train.values)
y_predF_100= modelF_100.predict(x_train)
PrediccionF_100=modelF_100.predict(x_test)
print("f1.score train: ",f1_score(y_true=y_train,y_pred=y_predF_100,average="micro"))
print("f1.score test: ",f1_score(y_true=y_test,y_pred=PrediccionF_100,average="micro"))

('f1.score train: ', 1.0)
('f1.score test: ', 0.68174846625766872)


In [24]:
from sklearn.ensemble import RandomForestClassifier
modelF_10 = RandomForestClassifier(n_estimators=10, max_depth=10000,n_jobs=-1)
modelF_10.set_params(warm_start=True,oob_score=True)
modelF_10.fit(x_train,y_train.values)
y_predF_10= modelF_10.predict(x_train)
PrediccionF_10=modelF_10.predict(x_test)
print("f1.score train: ",f1_score(y_true=y_train,y_pred=y_predF_10,average="micro"))
print("f1.score test: ",f1_score(y_true=y_test,y_pred=PrediccionF_10,average="micro"))

/home/maldos/anaconda2/lib/python2.7/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/home/maldos/anaconda2/lib/python2.7/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


('f1.score train: ', 0.99017908723281345)
('f1.score test: ', 0.64340490797546013)


In [25]:
from sklearn.ensemble import RandomForestClassifier
modelF_1= RandomForestClassifier(n_estimators=1, max_depth=10000,n_jobs=-1)
modelF_1.set_params(warm_start=True,oob_score=True)
modelF_1.fit(x_train,y_train.values)
y_predF_1= modelF_1.predict(x_train)
PrediccionF_1=modelF_1.predict(x_test)
print("f1.score train: ",f1_score(y_true=y_train,y_pred=y_predF_1,average="micro"))
print("f1.score test: ",f1_score(y_true=y_test,y_pred=PrediccionF_1,average="micro"))

('f1.score train: ', 0.82938571153475826)
('f1.score test: ', 0.51993865030674846)


En este caso el n estimator más efectivo es n=100. 

>g) Verifique que el OOB error (out of bag error) de los ensambladores que utilizan la técnica boostrap puede ser una alternativa como métrica de generalización, compárelo con el error calculado sobre el conjunto de pruebas y validación (o en su defecto cross validation).

In [26]:

oob_error = 1 - modelF_100.oob_score_
test_error = 1- modelF_100.score(x_test,y_test)
#val_error = 1- model_forest.score(x_val,y_val)
print("OOB error: ",oob_error)
#print ("Val error: ",val_error)
print("f1.score test: ",f1_score(y_true=y_test,y_pred=PrediccionF_100,average="micro"))

('OOB error: ', 0.31908338147506254)
('f1.score test: ', 0.68174846625766872)


El OOB representa el error de los subconjuntos con los datos no ingresados a este, y al compararlo con el test error podemos darnos cuenta como aporta a disminuir el error el usar muchos subconjuntos distintos.

>h) Entrene alguna otra máquina de aprendizaje, elegida por usted de entre todas las vistas en el curso, para resolver el problema. Elija los hiper-parámetros que estime convenientes intentando aumentar el F1-score obtenido por los algoritmos anteriores ¿Se logra una mejora? ¿Por qué?

Para resolver este problema usaremos un simple regresor lineal y diremos que su clasificación es la clase aplicando la función techo.

In [27]:
import math
from sklearn.linear_model import LinearRegression
model_lineal = LinearRegression(fit_intercept=True, normalize=False)
model_lineal.fit(x_train, y_train.values)
y_predlin = model_lineal.predict(x_train)
for i in range(len(y_predlin)):
    if ( (y_predlin[i] - math.floor(y_predlin[i])) > 0.5 ):
        y_predlin[i] = math.ceil(y_predlin[i])
    else:
        y_predlin[i] = math.floor(y_predlin[i])
Prediccionlin=model_lineal.predict(x_test)
for i in range(len(Prediccionlin)):
    if ( (Prediccionlin[i] - math.floor(Prediccionlin[i])) > 0.5 ):
        Prediccionlin[i] = math.ceil(Prediccionlin[i])
    else:
        Prediccionlin[i] = math.floor(Prediccionlin[i])
print("f1.score train: ",f1_score(y_true=y_train,y_pred=y_predlin,average="micro"))
print("f1.score test: ",f1_score(y_true=y_test,y_pred=Prediccionlin,average="micro"))

('f1.score train: ', 0.53610629693818601)
('f1.score test: ', 0.51380368098159512)


Una simple regresión lineal no puede determinar el ranking aproximado de calidad de un vino. Pero para poder tener un mejor resultado luego de obtener su puntuación continua es encasillarla en función de su grado para que se predigan valores discretos por medio de la función piso o techo aplicadas.
EJ: El modelo de regresión lineal predice un valor para "y" continuo tal que "y" quiere tomar el valor 4.3
debemos discretizar el valor 4.3 por medio de la función piso para finalmente indicar que el vino tiene calidad 4.

>i) Compare y analice las distintas maneras con las que se resolvió el problema definido en b), por ejemplo incluya las decisiones que conlleva y los resultados que reflejan.

In [28]:
print "acuraccy\t\tmethod"
print str(f1_score(y_true=y_test,y_pred=PrediccionT,average="micro"))+"\t"+"Tree"
print str(f1_score(y_true=y_test,y_pred=PrediccionB_1000,average="micro"))+"\t"+"Bagging n-estimator = 1000"
print str(f1_score(y_true=y_test,y_pred=PrediccionB_100,average="micro"))+"\t"+"Bagging n-estimator = 100"
print str(f1_score(y_true=y_test,y_pred=PrediccionB_10,average="micro"))+"\t"+"Bagging n-estimator = 10"
print str(f1_score(y_true=y_test,y_pred=PrediccionB_1,average="micro"))+"\t"+"Bagging n-estimator = 1"
print str(f1_score(y_true=y_test,y_pred=PrediccionA_1000,average="micro"))+"\t"+"AdaBoost n-estimator = 1000"
print str(f1_score(y_true=y_test,y_pred=PrediccionA_100,average="micro"))+"\t"+"AdaBoost n-estimator = 100"
print str(f1_score(y_true=y_test,y_pred=PrediccionA_10,average="micro"))+"\t"+"AdaBoost n-estimator = 10"
print str(f1_score(y_true=y_test,y_pred=PrediccionA_1,average="micro"))+"\t"+"AdaBoost n-estimator = 1"
print str(f1_score(y_true=y_test,y_pred=PrediccionF_1000,average="micro"))+"\t"+"Random forest n-estimator = 1000"
print str(f1_score(y_true=y_test,y_pred=PrediccionF_100,average="micro"))+"\t"+"Random forest n-estimator = 100"
print str(f1_score(y_true=y_test,y_pred=PrediccionF_10,average="micro"))+"\t"+"Random forest n-estimator = 10"
print str(f1_score(y_true=y_test,y_pred=PrediccionF_1,average="micro"))+"\t"+"Random forest n-estimator = 1"
print str(f1_score(y_true=y_test,y_pred=Prediccionlin,average="micro"))+"\t"+"Linear regression"

acuraccy		method
0.603527607362	Tree
0.681748466258	Bagging n-estimator = 1000
0.683282208589	Bagging n-estimator = 100
0.654141104294	Bagging n-estimator = 10
0.570552147239	Bagging n-estimator = 1
0.605828220859	AdaBoost n-estimator = 1000
0.605828220859	AdaBoost n-estimator = 100
0.608128834356	AdaBoost n-estimator = 10
0.60736196319	AdaBoost n-estimator = 1
0.686349693252	Random forest n-estimator = 1000
0.681748466258	Random forest n-estimator = 100
0.643404907975	Random forest n-estimator = 10
0.519938650307	Random forest n-estimator = 1
0.513803680982	Linear regression


El mejor de los modelos es el random forest con n-estimator=100 seguido por el bagging con nestimator=1000, lo cual no nos sorprende, pero esperabamos que el adaboost tuviera mejor resultado.

>j) Defina otra forma de combinar los valores que entregan los ensamblados al hacer predicciones y compare con lo que se hace actualmente, por ejemplo Bagging realiza el voto de la mayoría para clasificación y promedio para regresión, AdaBoost realiza una combinación ponderada de cada clasificador dependiendo de su habilidad (desempeño para clasificar el conjunto de entrenamiento).

In [29]:
list_estimator=[]
list_estimator.append(modelF_100)
list_estimator.append(modelF_1000)
list_estimator.append(modelB_1000)
list_predictions = [estimator.predict(x_test) for estimator in list_estimator]

In [30]:
list_predictions

[array([5, 5, 6, ..., 6, 6, 6]),
 array([5, 6, 6, ..., 6, 6, 6]),
 array([5, 6, 6, ..., 6, 6, 6])]

In [31]:
for i in range(len(list_predictions[0])):
    if ( (list_predictions[0][i] - math.floor(list_predictions[0][i])) > 0.5 ):
        list_predictions[0][i] = math.ceil(list_predictions[0][i])
    else:
        list_predictions[0][i] = math.floor(list_predictions[0][i])

In [32]:
list_predictions

[array([5, 5, 6, ..., 6, 6, 6]),
 array([5, 6, 6, ..., 6, 6, 6]),
 array([5, 6, 6, ..., 6, 6, 6])]

In [33]:
print len(list_predictions[0])
print len(list_predictions[1])
print len(list_predictions[2])

1304
1304
1304


In [34]:
solucion=[]
for i in range(len(list_predictions[0])):
    count=[0,0,0,0,0,0,0,0,0,0,0];
    count[int(list_predictions[0][i])] +=1
    count[list_predictions[1][i]] +=1
    count[list_predictions[2][i]] +=1
    solucion.append(count.index(max(count)))  


In [35]:
print("f1.score: ",f1_score(y_true=y_test,y_pred=solucion,average="micro"))

('f1.score: ', 0.68634969325153372)


Se puede apreciar que al combinar los mejores modelos, se supera el mejor de ellos,por lo cual si vale la pena combinar para mejorar el f1.score.

>k) Utilice la técnica de ensamblado para seleccionar características, para ésto defina un criterio para estimar la importancia de los distintos atributos en el ensamblado, impleméntelo sobre alguno de los ensambladores entrenados para resolver el problema definido en b). Realice un ranking de importancia de atributos y seleccione las  𝑘  características más relevantes.

In [47]:
modelF_1000 = RandomForestClassifier(n_estimators=1000, max_depth=10000,n_jobs=-1)
modelF_1000.set_params(warm_start=True,oob_score=True)
modelF_1000.fit(x_train,y_train.values)

print("f1.score train: ",f1_score(y_true=y_train,y_pred=y_predF_1000,average="micro"))
print("f1.score test: ",f1_score(y_true=y_test,y_pred=PrediccionF_1000,average="micro"))

y_pred = modelF_1000.predict(x_test)
D1000= f1_score(y_test, y_pred, average=None)

('f1.score train: ', 1.0)
('f1.score test: ', 0.68634969325153372)


In [37]:
import pandas as pd
feature_importances = pd.DataFrame(
                        modelF_1000.feature_importances_,
                        index = x_train.columns,
                        columns=['importance']
                        ).sort_values('importance', ascending=False)

In [38]:
feature_importances

,importance
alcohol,0.123006
density,0.102088
volatile acidity,0.098506
total sulfur dioxide,0.088940
chlorides,0.086792
sulphates,0.086580
free sulfur dioxide,0.085363
residual sugar,0.084608
pH,0.084254
citric acid,0.079150


Si fuera posible esto se podria comparar con la opinion de un experto para ver que tan alejado estamos de la realidad y podemos ver que las variables red y white no influyen mucho, esto es debido a que son dependientes entre ellas y que un vino puede ser malo o bueno sin importar de que tipo sea(tinto o blanco).

>l) Entrene la máquina de aprendizaje definida en h) sobre las  𝑘  carecterísticas derivadas del punto anterior ¿Mejora los resultados sobre ésta máquina de aprendizaje?

In [49]:
x_train2 = x_train.drop(columns=['red',
                                ])
x_test2  = x_test.drop(columns=['red',
                                ])


model2F_100 = RandomForestClassifier(n_estimators=100, max_depth=10000,n_jobs=-1)
model2F_100.set_params(warm_start=True,oob_score=True)
model2F_100.fit(x_train2,y_train.values)
Prediccion2F_100=model2F_100.predict(x_test2)
print("f1.score test sin drop: ",f1_score(y_true=y_test,y_pred=PrediccionF_100,average="micro"))
print("f1.score test con drop: ",f1_score(y_true=y_test,y_pred=Prediccion2F_100,average="micro"))

('f1.score test sin drop: ', 0.68174846625766872)
('f1.score test con drop: ', 0.68251533742331283)


In [45]:
print 0.68251533742331283 > 0.68174846625766872

True


Al quitar k etiquetas desde las menos relevantes a las más relevantes en general no mejoran la predicción del modelo de regresión lineal propuesto en H ya que cada característica es capaz de sumar o definir en cierta medida la calidad del vino. Sin embargo fué posible apreciar una etiqueta mal definida durante la definición de la representación del problema ya que se asignaron columnas "red" y "white" en las que se indicaba un 1 o un 0 si el vino correspondía a alguna de esas clases. Sin embargo fue posible mejorar el modelo en una pequeña medida al quitar la columna "red" y dejar solo la columna "white". Lo anterior cambia la representación ya que ahora en la columna "white" se indicará con 1 si el vino es blanco y con 0 si el vino es red, esto se debia a la dependencia de las dos variables.